In [ ]:
using Graphs
using Plots
using LinearAlgebra
using Statistics
include("../sis.jl")
include("../mpdbp.jl")
include("../exact/montecarlo.jl")
include("../glauber.jl")
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [ ]:
q = q_sis
T = 40
N = 12
k = 3
gg = random_regular_graph(N, k)
g = IndexedGraph(gg)
λ = 0.4
κ = 0.4
p⁰ = map(1:N) do i
    r = 0.95
    [r, 1-r]
end
ϕ = [[ones(2) for t in 1:T] for i in 1:N]

# throw random observations
nobs_node = 10
its = rand(collect(Iterators.product(1:N, 1:T)), nobs_node)
for (i, t) in its
    r = rand()
    ϕ[i][t] .= [r, 1-r]
end

# throw random pair observations
nobs_edge = 10
its = rand(collect(Iterators.product(edges(g), 1:T)), nobs_edge)
O = map(its) do (e,t)
    i, j, ij = e
    (i, j, t, rand(2,2))
end

ψ = pair_observations_directed(O, IndexedBiDiGraph(gg), T, q)

sis = SIS(g, λ, κ, p⁰, ϕ, ψ);

In [ ]:
bp = mpdbp(sis, d=1)
cb = CB_BP(bp);

In [ ]:
svd_trunc = TruncThresh(1e-2)
svd_trunc = TruncBond(4)
iters, cb = iterate!(bp, maxiter=100; svd_trunc, cb, tol=1e-3);

In [ ]:
pl_err = plot(cb.Δs, ylabel="convergence error", xlabel="iters", m=:o, yaxis=:log10, 
    size=(300,200), label="")

In [ ]:
b_bp = beliefs(bp)
p_bp = [[bbb[2] for bbb in bb] for bb in b_bp];

In [ ]:
ns = 10^5
sms = sample(bp, ns)
b_mc = marginals(sms)
p_mc = [[bbb[2] for bbb in bb] for bb in b_mc];

In [ ]:
pl_mc = plot(xlabel="time", ylabel="\$P(x_i=I)\$", xticks=0:5:T, title="Monte Carlo $ns samples")
for i in 1:N
    plot!(pl_mc, 0:T, p_mc[i], label="i=$i", m=:o, ms=3, lw=1)
end
pl_bp = plot(xlabel="time", ylabel="\$P(x_i=I)\$", xticks=0:5:T, title="MPdBP")
for i in 1:N
    plot!(pl_bp, 0:T, p_bp[i], label="i=$i", m=:o, ms=3, lw=1)
end
pl_sc = scatter(reduce(vcat, p_bp), reduce(vcat, p_mc), xlabel="MPdBP", ylabel="MonteCarlo", 
    label="\$P(x_i=I)\$", ms=3, c=:black, legend=:outertopright)
plot!(identity, label="", size=(300,300))
println("SIS on random regular graph N=$N, k=$k. P(S→I)=$λ, P(I→S)=$κ.")
println("$nobs_node random node observations, $nobs_edge random edge observations")
println("Max matrix size: ", svd_trunc.mprime, "x", svd_trunc.mprime)
plot(pl_mc, pl_bp, pl_sc, titlefontsize=10, size=(950, 300), legend=:outertopright, 
    margin=5Plots.mm, layout=(1,3))